In [1]:
fff=!pwd
print(fff[0]+"/analysis_expression_for_train.ipynb")

/workspace/02_cha_drug_repositioning/analysis_expression_for_train.ipynb


In [2]:
%load_ext autoreload
%autoreload 2

# Imports

In [3]:
import numpy as np
import pandas as pd
import copy
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
import pickle

from cmapPy.pandasGEXpress.parse import parse
from cmapPy.pandasGEXpress.write_gct import write
from cmapPy.pandasGEXpress.GCToo import GCToo
from cmapPy.pandasGEXpress.gctx2gct import gctx2gct_main

import torch

import warnings
warnings.filterwarnings('ignore')

/root/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from super_classes import param_class as param_class_module
from utils import data_processing as data_processing_module
from utils import data_load as data_load_module
from run import train as train_module
from run import test_model as test_model_module

# Configure parameters

In [5]:
BASE_DIR="/workspace/datafiles/2023/04/02_cha_drug_repositioning/paper_data"
GCTX_PATH_GSE70138="GSE70138_Broad_LINCS_Level4_ZSPCINF_mlr12k_n345976x12328_2017-03-06.gctx"
GCTX_PATH_GSE92742="GSE92742_Broad_LINCS_Level4_ZSPCINF_mlr12k_n1319138x12328.gctx"
TXT_META_PATH_GSE70138="GSE70138_Broad_LINCS_inst_info_2017-03-06.txt"
TXT_META_PATH_GSE92742="GSE92742_Broad_LINCS_inst_info.txt"
PERT_META_PATH_GSE70138="GSE70138_Broad_LINCS_pert_info_2017-03-06.txt"
PERT_META_PATH_GSE92742="GSE92742_Broad_LINCS_pert_info.txt"
TXT_LANDMARK_GENES_PATH="GSE92742_Broad_LINCS_gene_info_delta_landmark.txt"

In [6]:
configurable_parameters_dict={
  "BASE_DIR":BASE_DIR,
  "GCTX_PATH_GSE70138":GCTX_PATH_GSE70138,
  "GCTX_PATH_GSE92742":GCTX_PATH_GSE92742,
  "TXT_META_PATH_GSE70138":TXT_META_PATH_GSE70138,
  "TXT_META_PATH_GSE92742":TXT_META_PATH_GSE92742,
  "PERT_META_PATH_GSE70138":PERT_META_PATH_GSE70138,
  "PERT_META_PATH_GSE92742":PERT_META_PATH_GSE92742
}

# Load data

In [7]:
loaded_gctx_GSE70138=data_load_module.Utils_Data_Load.load_gctx(BASE_DIR,GCTX_PATH_GSE70138)
loaded_gctx_GSE92742=data_load_module.Utils_Data_Load.load_gctx(BASE_DIR,GCTX_PATH_GSE92742)

In [14]:
display(loaded_gctx_GSE70138.head(2))
print(loaded_gctx_GSE70138.shape)

cid,REP.A001_A375_24H_X1_B22:A03,REP.A001_A375_24H_X1_B22:A04,REP.A001_A375_24H_X1_B22:A05,REP.A001_A375_24H_X1_B22:A06,REP.A001_A375_24H_X1_B22:A07,REP.A001_A375_24H_X1_B22:A08,REP.A001_A375_24H_X1_B22:A09,REP.A001_A375_24H_X1_B22:A10,REP.A001_A375_24H_X1_B22:A11,REP.A001_A375_24H_X1_B22:A12,...,XPR002_YAPC.311_96H_X3_B22:P14,XPR002_YAPC.311_96H_X3_B22:P15,XPR002_YAPC.311_96H_X3_B22:P16,XPR002_YAPC.311_96H_X3_B22:P17,XPR002_YAPC.311_96H_X3_B22:P18,XPR002_YAPC.311_96H_X3_B22:P19,XPR002_YAPC.311_96H_X3_B22:P21,XPR002_YAPC.311_96H_X3_B22:P22,XPR002_YAPC.311_96H_X3_B22:P23,XPR002_YAPC.311_96H_X3_B22:P24
rid,,,,,,,,,,,,,,,,,,,,,
780,14.8204,-0.5200,-1.0369,-0.3005,0.4163,-0.1698,-0.3789,0.0907,0.5457,-1.1748,...,0.9126,0.1848,0.0567,0.2624,-0.8600,3.7718,-0.0917,0.1214,-1.2161,-0.1781
7849,0.0000,4.3134,-1.1246,-0.4670,-0.2225,-0.1234,-0.3441,-0.9418,1.2708,-2.1935,...,0.2852,-0.4669,0.1521,0.1521,0.0461,-0.1812,-0.6319,1.6618,0.7115,0.3420


(12328, 345976)


In [12]:
display(loaded_gctx_GSE92742.head(2))
print(loaded_gctx_GSE92742.shape)

cid,CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,CPC005_A375_6H_X3_B3_DUO52HI53LO:K06,CPC005_A375_6H_X1_B3_DUO52HI53LO:C19,CPC005_A375_6H_X2_B3_DUO52HI53LO:C19,CPC005_A375_6H_X3_B3_DUO52HI53LO:C19,CPC004_A375_6H_X1_B3_DUO52HI53LO:K13,CPC004_A375_6H_X2_B3_DUO52HI53LO:K13,CPC004_A375_6H_X3_B3_DUO52HI53LO:K13,CPC005_A375_6H_X1_B3_DUO52HI53LO:K20,...,PCLB003_PC3_24H_X3_B13:P15,PCLB003_PC3_24H_X3_B13:P16,PCLB003_PC3_24H_X3_B13:P17,PCLB003_PC3_24H_X3_B13:P18,PCLB003_PC3_24H_X3_B13:P19,PCLB003_PC3_24H_X3_B13:P20,PCLB003_PC3_24H_X3_B13:P21,PCLB003_PC3_24H_X3_B13:P22,PCLB003_PC3_24H_X3_B13:P23,PCLB003_PC3_24H_X3_B13:P24
rid,,,,,,,,,,,,,,,,,,,,,
5720,0.8509,0.6964,0.7763,-0.7216,-0.5528,-0.8905,1.1575,-10.4761,-0.2220,-0.4631,...,-2.7870,-0.9472,-1.2010,0.1184,2.1614,0.4863,-0.0174,1.3497,0.7177,0.6071
466,0.0117,-1.1202,-1.2383,0.2762,-1.9274,-1.1556,1.6463,2.9997,-0.1231,-0.9180,...,4.7663,4.8015,1.3907,0.8861,-2.5416,4.4342,-1.2144,4.7777,0.6917,0.2969


(12328, 1319138)


# Clean column and row names

In [15]:
loaded_gctx_GSE70138=data_processing_module.Utils_Data_Processing.clean_column_row_names(loaded_gctx_GSE70138)
loaded_gctx_GSE92742=data_processing_module.Utils_Data_Processing.clean_column_row_names(loaded_gctx_GSE92742)

In [16]:
display(loaded_gctx_GSE70138.head(2))
display(loaded_gctx_GSE92742.head(2))

,REP.A001_A375_24H_X1_B22:A03,REP.A001_A375_24H_X1_B22:A04,REP.A001_A375_24H_X1_B22:A05,REP.A001_A375_24H_X1_B22:A06,REP.A001_A375_24H_X1_B22:A07,REP.A001_A375_24H_X1_B22:A08,REP.A001_A375_24H_X1_B22:A09,REP.A001_A375_24H_X1_B22:A10,REP.A001_A375_24H_X1_B22:A11,REP.A001_A375_24H_X1_B22:A12,...,XPR002_YAPC.311_96H_X3_B22:P14,XPR002_YAPC.311_96H_X3_B22:P15,XPR002_YAPC.311_96H_X3_B22:P16,XPR002_YAPC.311_96H_X3_B22:P17,XPR002_YAPC.311_96H_X3_B22:P18,XPR002_YAPC.311_96H_X3_B22:P19,XPR002_YAPC.311_96H_X3_B22:P21,XPR002_YAPC.311_96H_X3_B22:P22,XPR002_YAPC.311_96H_X3_B22:P23,XPR002_YAPC.311_96H_X3_B22:P24
780,14.8204,-0.5200,-1.0369,-0.3005,0.4163,-0.1698,-0.3789,0.0907,0.5457,-1.1748,...,0.9126,0.1848,0.0567,0.2624,-0.8600,3.7718,-0.0917,0.1214,-1.2161,-0.1781
7849,0.0000,4.3134,-1.1246,-0.4670,-0.2225,-0.1234,-0.3441,-0.9418,1.2708,-2.1935,...,0.2852,-0.4669,0.1521,0.1521,0.0461,-0.1812,-0.6319,1.6618,0.7115,0.3420


,CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,CPC005_A375_6H_X3_B3_DUO52HI53LO:K06,CPC005_A375_6H_X1_B3_DUO52HI53LO:C19,CPC005_A375_6H_X2_B3_DUO52HI53LO:C19,CPC005_A375_6H_X3_B3_DUO52HI53LO:C19,CPC004_A375_6H_X1_B3_DUO52HI53LO:K13,CPC004_A375_6H_X2_B3_DUO52HI53LO:K13,CPC004_A375_6H_X3_B3_DUO52HI53LO:K13,CPC005_A375_6H_X1_B3_DUO52HI53LO:K20,...,PCLB003_PC3_24H_X3_B13:P15,PCLB003_PC3_24H_X3_B13:P16,PCLB003_PC3_24H_X3_B13:P17,PCLB003_PC3_24H_X3_B13:P18,PCLB003_PC3_24H_X3_B13:P19,PCLB003_PC3_24H_X3_B13:P20,PCLB003_PC3_24H_X3_B13:P21,PCLB003_PC3_24H_X3_B13:P22,PCLB003_PC3_24H_X3_B13:P23,PCLB003_PC3_24H_X3_B13:P24
5720,0.8509,0.6964,0.7763,-0.7216,-0.5528,-0.8905,1.1575,-10.4761,-0.2220,-0.4631,...,-2.7870,-0.9472,-1.2010,0.1184,2.1614,0.4863,-0.0174,1.3497,0.7177,0.6071
466,0.0117,-1.1202,-1.2383,0.2762,-1.9274,-1.1556,1.6463,2.9997,-0.1231,-0.9180,...,4.7663,4.8015,1.3907,0.8861,-2.5416,4.4342,-1.2144,4.7777,0.6917,0.2969


# Load meta data

In [17]:
loaded_pert_txt_GSE70138=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,TXT_META_PATH_GSE70138)
loaded_pert_txt_GSE92742=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,TXT_META_PATH_GSE92742)
loaded_pert_actual_txt_GSE70138=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,PERT_META_PATH_GSE70138)
loaded_pert_actual_txt_GSE92742=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,PERT_META_PATH_GSE92742)
loaded_landmark_txt=data_load_module.Utils_Data_Load.load_txt(BASE_DIR,TXT_LANDMARK_GENES_PATH)

# left join "loaded_pert_txt" and "loaded_pert_actual_txt"

In [18]:
joint_data_pert_pert_actual_GSE70138=loaded_pert_txt_GSE70138.merge(loaded_pert_actual_txt_GSE70138.drop(columns=['pert_iname', 'pert_type']),left_on='pert_id',right_on='pert_id',how='left')
joint_data_pert_pert_actual_GSE92742=loaded_pert_txt_GSE92742.merge(loaded_pert_actual_txt_GSE92742.drop(columns=['pert_iname', 'pert_type']),left_on='pert_id',right_on='pert_id',how='left')

In [19]:
display(joint_data_pert_pert_actual_GSE70138.head(2))
display(joint_data_pert_pert_actual_GSE92742.head(2))

,inst_id,cell_id,det_plate,det_well,pert_mfc_id,pert_dose,pert_dose_unit,pert_id,pert_iname,pert_type,pert_time,pert_time_unit,canonical_smiles,inchi_key
0,LJP005_A375_24H_X1_B19:A03,A375,LJP005_A375_24H_X1_B19,A03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,24.0,h,CS(=O)C,IAZDPXIOMUYVGZ-UHFFFAOYSA-N
1,LPROT001_A375_6H_X1_B20:B03,A375,LPROT001_A375_6H_X1_B20,B03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,6.0,h,CS(=O)C,IAZDPXIOMUYVGZ-UHFFFAOYSA-N


,inst_id,rna_plate,rna_well,pert_id,pert_iname,pert_type,pert_dose,pert_dose_unit,pert_time,pert_time_unit,cell_id,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
0,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13,ASG001_MCF7_24H_X1,F13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666
1,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13,ASG001_MCF7_24H_X1,G13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666


# Set index data

In [20]:
joint_data_pert_pert_actual_GSE70138.index=list(joint_data_pert_pert_actual_GSE70138['inst_id'])
joint_data_pert_pert_actual_GSE92742.index=list(joint_data_pert_pert_actual_GSE92742['inst_id'])
loaded_landmark_txt.index=[str(i) for i in loaded_landmark_txt['pr_gene_id']]

In [21]:
display(joint_data_pert_pert_actual_GSE70138.head(2))
display(joint_data_pert_pert_actual_GSE92742.head(2))
display(loaded_landmark_txt.head(2))

,inst_id,cell_id,det_plate,det_well,pert_mfc_id,pert_dose,pert_dose_unit,pert_id,pert_iname,pert_type,pert_time,pert_time_unit,canonical_smiles,inchi_key
LJP005_A375_24H_X1_B19:A03,LJP005_A375_24H_X1_B19:A03,A375,LJP005_A375_24H_X1_B19,A03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,24.0,h,CS(=O)C,IAZDPXIOMUYVGZ-UHFFFAOYSA-N
LPROT001_A375_6H_X1_B20:B03,LPROT001_A375_6H_X1_B20:B03,A375,LPROT001_A375_6H_X1_B20,B03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,6.0,h,CS(=O)C,IAZDPXIOMUYVGZ-UHFFFAOYSA-N


,inst_id,rna_plate,rna_well,pert_id,pert_iname,pert_type,pert_dose,pert_dose_unit,pert_time,pert_time_unit,cell_id,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13,ASG001_MCF7_24H_X1,F13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13,ASG001_MCF7_24H_X1,G13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666


,pr_gene_id,pr_gene_symbol,pr_gene_title
780,780,DDR1,discoidin domain receptor tyrosine kinase 1
7849,7849,PAX8,paired box 8


# Remove -666 rows (non drugs)

In [22]:
joint_data_pert_pert_actual_remove_non_drug_GSE70138=joint_data_pert_pert_actual_GSE70138.drop(joint_data_pert_pert_actual_GSE70138[(joint_data_pert_pert_actual_GSE70138['canonical_smiles']=="-666")].index)
joint_data_pert_pert_actual_remove_non_drug_GSE92742=joint_data_pert_pert_actual_GSE92742.drop(joint_data_pert_pert_actual_GSE92742[(joint_data_pert_pert_actual_GSE92742['canonical_smiles']=="-666")].index)

In [23]:
display(joint_data_pert_pert_actual_remove_non_drug_GSE70138.head(2))
display(joint_data_pert_pert_actual_remove_non_drug_GSE92742.head(2))

,inst_id,cell_id,det_plate,det_well,pert_mfc_id,pert_dose,pert_dose_unit,pert_id,pert_iname,pert_type,pert_time,pert_time_unit,canonical_smiles,inchi_key
LJP005_A375_24H_X1_B19:A03,LJP005_A375_24H_X1_B19:A03,A375,LJP005_A375_24H_X1_B19,A03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,24.0,h,CS(=O)C,IAZDPXIOMUYVGZ-UHFFFAOYSA-N
LPROT001_A375_6H_X1_B20:B03,LPROT001_A375_6H_X1_B20:B03,A375,LPROT001_A375_6H_X1_B20,B03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,6.0,h,CS(=O)C,IAZDPXIOMUYVGZ-UHFFFAOYSA-N


,inst_id,rna_plate,rna_well,pert_id,pert_iname,pert_type,pert_dose,pert_dose_unit,pert_time,pert_time_unit,cell_id,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:F13,ASG001_MCF7_24H_X1,F13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:G13,ASG001_MCF7_24H_X1,G13,DMSO,DMSO,ctl_vehicle,0.1,%,24,h,MCF7,1.0,IAZDPXIOMUYVGZ,IAZDPXIOMUYVGZ-UHFFFAOYSA-N,CS(=O)C,-666


# Leave only trt_cp in pert_type

In [24]:
# https://clue.io/connectopedia/perturbagen_types_and_controls
joint_data_pert_pert_actual_remove_non_drug_GSE70138=joint_data_pert_pert_actual_remove_non_drug_GSE70138[joint_data_pert_pert_actual_remove_non_drug_GSE70138['pert_type']=="trt_cp"]
joint_data_pert_pert_actual_remove_non_drug_GSE92742=joint_data_pert_pert_actual_remove_non_drug_GSE92742[joint_data_pert_pert_actual_remove_non_drug_GSE92742['pert_type']=="trt_cp"]

In [26]:
display(joint_data_pert_pert_actual_remove_non_drug_GSE70138.head(2))
print(joint_data_pert_pert_actual_remove_non_drug_GSE70138.shape)
display(joint_data_pert_pert_actual_remove_non_drug_GSE92742.head(2))
print(joint_data_pert_pert_actual_remove_non_drug_GSE92742.shape)

,inst_id,cell_id,det_plate,det_well,pert_mfc_id,pert_dose,pert_dose_unit,pert_id,pert_iname,pert_type,pert_time,pert_time_unit,canonical_smiles,inchi_key
LJP005_A375_24H_X1_B19:A12,LJP005_A375_24H_X1_B19:A12,A375,LJP005_A375_24H_X1_B19,A12,BRD-K76908866-001-06-8,0.04,um,BRD-K76908866,CP-724714,trt_cp,24.0,h,COCC(=O)NC\C=C\c1ccc2ncnc(Nc3ccc(Oc4ccc(C)nc4)...,LLVZBTWPGQVVLW-SNAWJCMRSA-N
LJP005_A375_24H_X1_B19:E09,LJP005_A375_24H_X1_B19:E09,A375,LJP005_A375_24H_X1_B19,E09,BRD-K93788137-001-03-3,1.11,um,BRD-K93788137,PF-431396,trt_cp,24.0,h,CN(c1ccccc1CNc1nc(Nc2ccc3NC(=O)Cc3c2)ncc1C(F)(...,POJZIZBONPAWIV-UHFFFAOYSA-N


(314422, 14)


,inst_id,rna_plate,rna_well,pert_id,pert_iname,pert_type,pert_dose,pert_dose_unit,pert_time,pert_time_unit,cell_id,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:A06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:A06,ASG001_MCF7_24H_X1,A06,BRD-A60070924,alpha-estradiol,trt_cp,10.0,um,24,h,MCF7,1.0,VOXZDWNPVJITMN,VOXZDWNPVJITMN-UHFFFAOYSA-N,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,450
ASG001_MCF7_24H_X1_B7_DUO52HI53LO:B06,ASG001_MCF7_24H_X1_B7_DUO52HI53LO:B06,ASG001_MCF7_24H_X1,B06,BRD-A60070924,alpha-estradiol,trt_cp,2.0,um,24,h,MCF7,1.0,VOXZDWNPVJITMN,VOXZDWNPVJITMN-UHFFFAOYSA-N,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,450


(663874, 16)


# Merge "loaded_gctx" and "loaded_landmark_txt" (leave only landmark genes)

In [27]:
expression_df_by_landmarks_GSE70138=pd.merge(loaded_gctx_GSE70138,loaded_landmark_txt,left_index=True,right_index=True,how='inner')
expression_df_by_landmarks_GSE92742=pd.merge(loaded_gctx_GSE92742,loaded_landmark_txt,left_index=True,right_index=True,how='inner')

In [29]:
display(expression_df_by_landmarks_GSE70138.head(2))
display(expression_df_by_landmarks_GSE92742.head(2))

,REP.A001_A375_24H_X1_B22:A03,REP.A001_A375_24H_X1_B22:A04,REP.A001_A375_24H_X1_B22:A05,REP.A001_A375_24H_X1_B22:A06,REP.A001_A375_24H_X1_B22:A07,REP.A001_A375_24H_X1_B22:A08,REP.A001_A375_24H_X1_B22:A09,REP.A001_A375_24H_X1_B22:A10,REP.A001_A375_24H_X1_B22:A11,REP.A001_A375_24H_X1_B22:A12,...,XPR002_YAPC.311_96H_X3_B22:P17,XPR002_YAPC.311_96H_X3_B22:P18,XPR002_YAPC.311_96H_X3_B22:P19,XPR002_YAPC.311_96H_X3_B22:P21,XPR002_YAPC.311_96H_X3_B22:P22,XPR002_YAPC.311_96H_X3_B22:P23,XPR002_YAPC.311_96H_X3_B22:P24,pr_gene_id,pr_gene_symbol,pr_gene_title
780,14.8204,-0.5200,-1.0369,-0.3005,0.4163,-0.1698,-0.3789,0.0907,0.5457,-1.1748,...,0.2624,-0.8600,3.7718,-0.0917,0.1214,-1.2161,-0.1781,780,DDR1,discoidin domain receptor tyrosine kinase 1
7849,0.0000,4.3134,-1.1246,-0.4670,-0.2225,-0.1234,-0.3441,-0.9418,1.2708,-2.1935,...,0.1521,0.0461,-0.1812,-0.6319,1.6618,0.7115,0.3420,7849,PAX8,paired box 8


,CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,CPC005_A375_6H_X3_B3_DUO52HI53LO:K06,CPC005_A375_6H_X1_B3_DUO52HI53LO:C19,CPC005_A375_6H_X2_B3_DUO52HI53LO:C19,CPC005_A375_6H_X3_B3_DUO52HI53LO:C19,CPC004_A375_6H_X1_B3_DUO52HI53LO:K13,CPC004_A375_6H_X2_B3_DUO52HI53LO:K13,CPC004_A375_6H_X3_B3_DUO52HI53LO:K13,CPC005_A375_6H_X1_B3_DUO52HI53LO:K20,...,PCLB003_PC3_24H_X3_B13:P18,PCLB003_PC3_24H_X3_B13:P19,PCLB003_PC3_24H_X3_B13:P20,PCLB003_PC3_24H_X3_B13:P21,PCLB003_PC3_24H_X3_B13:P22,PCLB003_PC3_24H_X3_B13:P23,PCLB003_PC3_24H_X3_B13:P24,pr_gene_id,pr_gene_symbol,pr_gene_title
55847,0.2106,1.1142,-0.5801,-0.9112,0.5033,3.0285,0.9118,0.0184,-0.0293,-1.2838,...,0.1329,0.4067,0.5962,-0.1673,1.3051,0.7453,0.1329,55847,CISD1,CDGSH iron sulfur domain 1
25803,0.2892,0.1883,0.6909,1.4429,6.0724,-1.2484,0.8752,3.4359,0.1169,-1.3850,...,-1.9368,-1.1910,0.1538,-0.9033,-0.9033,-1.6760,0.5652,25803,SPDEF,SAM pointed domain containing ETS transcriptio...


# Transpose df

In [30]:
expression_df_by_landmarks_T_GSE70138=expression_df_by_landmarks_GSE70138.T
expression_df_by_landmarks_T_GSE92742=expression_df_by_landmarks_GSE92742.T

In [31]:
display(expression_df_by_landmarks_T_GSE70138.head(2))
display(expression_df_by_landmarks_T_GSE92742.head(2))

,780,7849,23,9552,10921,10285,533,23326,9761,3927,...,1497,7264,23523,2767,23038,57787,57048,84619,5526,79716
REP.A001_A375_24H_X1_B22:A03,14.8204,0.0,-0.3428,-0.7251,0.0,0.2381,0.3615,-0.2455,-1.4629,-0.3523,...,0.7094,0.375,1.9697,1.0853,-0.672,0.1111,4.2066,-1.8642,1.8391,-0.8088
REP.A001_A375_24H_X1_B22:A04,-0.52,4.3134,-0.8369,0.2381,0.0,0.8633,0.9437,1.345,0.9622,-0.4702,...,0.867,0.484,-0.3556,1.5125,0.8923,-0.5714,-0.3861,-0.35,-0.9354,-0.7052


,55847,25803,3480,2946,9517,25987,23670,11182,2146,3385,...,5526,25836,5523,1837,1955,22930,7840,3046,55278,4122
CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,0.2106,0.2892,-0.3222,-0.5751,-0.8432,-0.0981,0.1181,0.3545,-0.9332,0.9282,...,-0.2673,0.5202,0.4783,0.3221,1.7838,0.6258,-0.6607,1.5576,0.5938,0.8408
CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,1.1142,0.1883,-0.0239,1.121,-0.8318,-0.4604,-0.7833,0.2543,-0.8127,-0.2484,...,-0.1253,-0.4552,1.8226,1.9999,1.1418,-1.5645,1.2444,1.9761,0.52,1.0644


# Give gene names to columns

In [32]:
expression_df_by_landmarks_T_GSE70138.columns=list(expression_df_by_landmarks_T_GSE70138.loc["pr_gene_symbol",:])
expression_df_by_landmarks_T_GSE92742.columns=list(expression_df_by_landmarks_T_GSE92742.loc["pr_gene_symbol",:])

In [33]:
display(expression_df_by_landmarks_T_GSE70138.head(2))
display(expression_df_by_landmarks_T_GSE92742.head(2))

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CTNS,TSTA3,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1
REP.A001_A375_24H_X1_B22:A03,14.8204,0.0,-0.3428,-0.7251,0.0,0.2381,0.3615,-0.2455,-1.4629,-0.3523,...,0.7094,0.375,1.9697,1.0853,-0.672,0.1111,4.2066,-1.8642,1.8391,-0.8088
REP.A001_A375_24H_X1_B22:A04,-0.52,4.3134,-0.8369,0.2381,0.0,0.8633,0.9437,1.345,0.9622,-0.4702,...,0.867,0.484,-0.3556,1.5125,0.8923,-0.5714,-0.3861,-0.35,-0.9354,-0.7052


,CISD1,SPDEF,IGF1R,GSTM2,SPTLC2,TSKU,TMEM2,SLC2A6,EZH2,ICAM3,...,PPP2R5B,NIPBL,PPP2R3A,DTNA,MEGF9,RAB3GAP1,ALMS1,HBE1,QRSL1,MAN2A2
CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,0.2106,0.2892,-0.3222,-0.5751,-0.8432,-0.0981,0.1181,0.3545,-0.9332,0.9282,...,-0.2673,0.5202,0.4783,0.3221,1.7838,0.6258,-0.6607,1.5576,0.5938,0.8408
CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,1.1142,0.1883,-0.0239,1.121,-0.8318,-0.4604,-0.7833,0.2543,-0.8127,-0.2484,...,-0.1253,-0.4552,1.8226,1.9999,1.1418,-1.5645,1.2444,1.9761,0.52,1.0644


In [39]:
test_keys=[str(one) for one in list(loaded_landmark_txt['pr_gene_id'])]
test_values=loaded_landmark_txt['pr_gene_symbol']
res = {test_keys[i]: test_values[i] for i in range(len(test_keys))}
list(res.items())[:4]

[('780', 'DDR1'), ('7849', 'PAX8'), ('23', 'ABCF1'), ('9552', 'SPAG7')]

# Remove unnecessary data (last 3 rows)

In [40]:
expression_df_by_landmarks_T_wo_unnecessary_rows_GSE70138=expression_df_by_landmarks_T_GSE70138.drop(["pr_gene_id","pr_gene_symbol","pr_gene_title"],axis='index')
expression_df_by_landmarks_T_wo_unnecessary_rows_GSE92742=expression_df_by_landmarks_T_GSE92742.drop(["pr_gene_id","pr_gene_symbol","pr_gene_title"],axis='index')

In [41]:
display(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE70138.head(2))
display(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE92742.head(2))

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CTNS,TSTA3,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1
REP.A001_A375_24H_X1_B22:A03,14.8204,0.0,-0.3428,-0.7251,0.0,0.2381,0.3615,-0.2455,-1.4629,-0.3523,...,0.7094,0.375,1.9697,1.0853,-0.672,0.1111,4.2066,-1.8642,1.8391,-0.8088
REP.A001_A375_24H_X1_B22:A04,-0.52,4.3134,-0.8369,0.2381,0.0,0.8633,0.9437,1.345,0.9622,-0.4702,...,0.867,0.484,-0.3556,1.5125,0.8923,-0.5714,-0.3861,-0.35,-0.9354,-0.7052


,CISD1,SPDEF,IGF1R,GSTM2,SPTLC2,TSKU,TMEM2,SLC2A6,EZH2,ICAM3,...,PPP2R5B,NIPBL,PPP2R3A,DTNA,MEGF9,RAB3GAP1,ALMS1,HBE1,QRSL1,MAN2A2
CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,0.2106,0.2892,-0.3222,-0.5751,-0.8432,-0.0981,0.1181,0.3545,-0.9332,0.9282,...,-0.2673,0.5202,0.4783,0.3221,1.7838,0.6258,-0.6607,1.5576,0.5938,0.8408
CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,1.1142,0.1883,-0.0239,1.121,-0.8318,-0.4604,-0.7833,0.2543,-0.8127,-0.2484,...,-0.1253,-0.4552,1.8226,1.9999,1.1418,-1.5645,1.2444,1.9761,0.52,1.0644


# Merge "expression_df_by_landmarks_T_wo_unnecessary_rows" and "joint_data_pert_pert_actual_remove_non_drug"

In [43]:
expression_pert_df_GSE70138=pd.merge(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE70138,joint_data_pert_pert_actual_remove_non_drug_GSE70138[['pert_iname']],left_index=True,right_index=True,how='inner')
expression_pert_df_GSE92742=pd.merge(expression_df_by_landmarks_T_wo_unnecessary_rows_GSE92742,joint_data_pert_pert_actual_remove_non_drug_GSE92742[['pert_iname']],left_index=True,right_index=True,how='inner')

In [44]:
display(expression_pert_df_GSE70138.head(2))
display(expression_pert_df_GSE92742.head(2))

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,TSTA3,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname
REP.A001_A375_24H_X1_B22:A07,0.4163,-0.2225,0.9273,0.0202,0.0,-0.4607,0.0204,0.6745,-0.3362,0.6315,...,-1.9697,-0.7345,3.5454,-7.9314,-1.4845,0.7707,0.2316,-0.8891,-0.1372,aminoguanidine
REP.A001_A375_24H_X1_B22:A08,-0.1698,-0.1234,-0.1034,0.464,-16.6213,-0.4256,0.751,1.345,-0.4169,-1.3208,...,-1.4121,-0.0553,1.4655,0.342,-2.3748,-0.4059,0.0063,-0.4077,13.4469,aminoguanidine


,CISD1,SPDEF,IGF1R,GSTM2,SPTLC2,TSKU,TMEM2,SLC2A6,EZH2,ICAM3,...,NIPBL,PPP2R3A,DTNA,MEGF9,RAB3GAP1,ALMS1,HBE1,QRSL1,MAN2A2,pert_iname
CPC005_A375_6H_X1_B3_DUO52HI53LO:K06,0.2106,0.2892,-0.3222,-0.5751,-0.8432,-0.0981,0.1181,0.3545,-0.9332,0.9282,...,0.5202,0.4783,0.3221,1.7838,0.6258,-0.6607,1.5576,0.5938,0.8408,quinpirole
CPC005_A375_6H_X2_B3_DUO52HI53LO:K06,1.1142,0.1883,-0.0239,1.121,-0.8318,-0.4604,-0.7833,0.2543,-0.8127,-0.2484,...,-0.4552,1.8226,1.9999,1.1418,-1.5645,1.2444,1.9761,0.52,1.0644,quinpirole


In [45]:
display(expression_pert_df_GSE70138.shape)
display(expression_pert_df_GSE92742.shape)

(314422, 978)

(663874, 978)

# Concatenate two LINCS data

In [46]:
expression_pert_df_GSE70138['LINCS']=["GSE70138"]*expression_pert_df_GSE70138.shape[0]
expression_pert_df_GSE92742['LINCS']=["GSE92742"]*expression_pert_df_GSE92742.shape[0]

In [47]:
expression_pert_df_cat=pd.concat((expression_pert_df_GSE70138,expression_pert_df_GSE92742),axis=0)

In [48]:
expression_pert_df_cat

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,0.4163,-0.2225,0.9273,0.0202,0.0,-0.4607,0.0204,0.6745,-0.3362,0.6315,...,-0.7345,3.5454,-7.9314,-1.4845,0.7707,0.2316,-0.8891,-0.1372,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,-0.1698,-0.1234,-0.1034,0.464,-16.6213,-0.4256,0.751,1.345,-0.4169,-1.3208,...,-0.0553,1.4655,0.342,-2.3748,-0.4059,0.0063,-0.4077,13.4469,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,-0.3789,-0.3441,0.3622,-4.0071,0.0,0.0,0.8806,0.0,0.2052,-1.216,...,0.926,0.3322,0.1071,-0.0438,0.8398,-0.9396,1.5787,12.1437,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,0.0907,-0.9418,-0.0818,-1.0441,0.0,-0.3292,1.5043,0.6745,-0.1755,0.252,...,0.517,0.4509,-0.5566,0.2643,-0.3026,0.816,0.2214,12.1766,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,0.5457,1.2708,0.1439,0.0958,0.0,0.9065,-0.7551,0.487,0.2947,0.4657,...,-0.6222,1.3378,-0.0955,-2.066,0.2742,-1.6494,-0.8301,13.1083,aminoguanidine,GSE70138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.3662,-2.1044,0.2024,0.8759,-0.9267,-0.0539,-0.1612,2.8925,0.372,-0.0475,...,-0.976,0.0881,-2.5475,0.4033,1.1583,0.6428,-1.2056,-0.2261,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.3775,-0.4531,-0.0109,-0.266,-0.1719,0.0985,-0.1459,-2.066,-0.375,-1.2346,...,-1.0873,-2.0991,-1.4825,2.1151,0.0,0.5136,1.1579,-0.087,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.5546,0.0,-0.3511,1.6628,-0.8406,-0.6109,-0.4939,-1.4338,0.0589,0.0729,...,-1.0419,-0.2673,-1.2113,-1.0933,1.1583,-0.3659,-0.1505,0.4603,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.3037,-0.666,1.176,0.5265,-1.0765,0.36,0.4682,7.2672,-0.4834,-0.5445,...,-0.9612,-2.7238,-0.7698,0.3769,0.2699,0.2177,-1.0949,0.3538,wortmannin,GSE92742


# Select frequent compounds (freq>100, 1170 compounds)

In [49]:
pert_counts_df=expression_pert_df_cat['pert_iname'].value_counts().to_frame()
pert_counts_df

,pert_iname
bortezomib,6290
MG-132,6289
vorinostat,4768
geldanamycin,3907
trichostatin-a,3692
...,...
BRD-K80564983,2
BRD-K10196668,1
BRD-K47471281,1
BRD-K82494568,1


In [52]:
criterion_num=40
selected_compounds=list(pert_counts_df[pert_counts_df['pert_iname']>criterion_num].index)    ;len(selected_compounds)

5883

# Leave samples associated with selected compounds

In [53]:
expression_freq_pert_df=expression_pert_df_cat[expression_pert_df_cat['pert_iname'].isin(selected_compounds)]
expression_freq_pert_df

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,CABIN1,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS
REP.A001_A375_24H_X1_B22:A07,0.4163,-0.2225,0.9273,0.0202,0.0,-0.4607,0.0204,0.6745,-0.3362,0.6315,...,-0.7345,3.5454,-7.9314,-1.4845,0.7707,0.2316,-0.8891,-0.1372,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A08,-0.1698,-0.1234,-0.1034,0.464,-16.6213,-0.4256,0.751,1.345,-0.4169,-1.3208,...,-0.0553,1.4655,0.342,-2.3748,-0.4059,0.0063,-0.4077,13.4469,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A09,-0.3789,-0.3441,0.3622,-4.0071,0.0,0.0,0.8806,0.0,0.2052,-1.216,...,0.926,0.3322,0.1071,-0.0438,0.8398,-0.9396,1.5787,12.1437,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A10,0.0907,-0.9418,-0.0818,-1.0441,0.0,-0.3292,1.5043,0.6745,-0.1755,0.252,...,0.517,0.4509,-0.5566,0.2643,-0.3026,0.816,0.2214,12.1766,aminoguanidine,GSE70138
REP.A001_A375_24H_X1_B22:A11,0.5457,1.2708,0.1439,0.0958,0.0,0.9065,-0.7551,0.487,0.2947,0.4657,...,-0.6222,1.3378,-0.0955,-2.066,0.2742,-1.6494,-0.8301,13.1083,aminoguanidine,GSE70138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.3662,-2.1044,0.2024,0.8759,-0.9267,-0.0539,-0.1612,2.8925,0.372,-0.0475,...,-0.976,0.0881,-2.5475,0.4033,1.1583,0.6428,-1.2056,-0.2261,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P21,0.3775,-0.4531,-0.0109,-0.266,-0.1719,0.0985,-0.1459,-2.066,-0.375,-1.2346,...,-1.0873,-2.0991,-1.4825,2.1151,0.0,0.5136,1.1579,-0.087,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P22,0.5546,0.0,-0.3511,1.6628,-0.8406,-0.6109,-0.4939,-1.4338,0.0589,0.0729,...,-1.0419,-0.2673,-1.2113,-1.0933,1.1583,-0.3659,-0.1505,0.4603,wortmannin,GSE92742
PCLB003_PC3_24H_X3_B13:P23,0.3037,-0.666,1.176,0.5265,-1.0765,0.36,0.4682,7.2672,-0.4834,-0.5445,...,-0.9612,-2.7238,-0.7698,0.3769,0.2699,0.2177,-1.0949,0.3538,wortmannin,GSE92742


# Remove BRDs

In [81]:
expression_freq_pert_df=expression_freq_pert_df[~expression_freq_pert_df['pert_iname'].str.contains('BRD-')]

# Convert pert_iname to number

In [82]:
keys = list(expression_freq_pert_df['pert_iname'].unique())
values = list(range(len(expression_freq_pert_df['pert_iname'].unique())))
dictionary = dict(zip(keys, values))

In [83]:
print(list(dictionary.items())[:5])

[('aminoguanidine', 0), ('pritelivir', 1), ('batimastat', 2), ('CX-516', 3), ('aminoglutethimide', 4)]


In [84]:
number_of_perturbagens=len(list(expression_freq_pert_df['pert_iname'].unique()))
configurable_parameters_dict['number_of_perturbagens']=number_of_perturbagens
number_of_perturbagens

3820

In [85]:
pert_iname_num=expression_freq_pert_df['pert_iname'].map(dictionary)

In [86]:
expression_freq_pert_df['pert_iname_num']=pert_iname_num

In [87]:
expression_freq_pert_df

,DDR1,PAX8,ABCF1,SPAG7,RNPS1,SMNDC1,ATP6V0B,USP22,MLEC,LASP1,...,GNA11,WDTC1,MARK4,PLSCR3,ZGPAT,PPP2R5B,NPEPL1,pert_iname,LINCS,pert_iname_num
REP.A001_A375_24H_X1_B22:A07,0.4163,-0.2225,0.9273,0.0202,0.0,-0.4607,0.0204,0.6745,-0.3362,0.6315,...,3.5454,-7.9314,-1.4845,0.7707,0.2316,-0.8891,-0.1372,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A08,-0.1698,-0.1234,-0.1034,0.464,-16.6213,-0.4256,0.751,1.345,-0.4169,-1.3208,...,1.4655,0.342,-2.3748,-0.4059,0.0063,-0.4077,13.4469,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A09,-0.3789,-0.3441,0.3622,-4.0071,0.0,0.0,0.8806,0.0,0.2052,-1.216,...,0.3322,0.1071,-0.0438,0.8398,-0.9396,1.5787,12.1437,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A10,0.0907,-0.9418,-0.0818,-1.0441,0.0,-0.3292,1.5043,0.6745,-0.1755,0.252,...,0.4509,-0.5566,0.2643,-0.3026,0.816,0.2214,12.1766,aminoguanidine,GSE70138,0
REP.A001_A375_24H_X1_B22:A11,0.5457,1.2708,0.1439,0.0958,0.0,0.9065,-0.7551,0.487,0.2947,0.4657,...,1.3378,-0.0955,-2.066,0.2742,-1.6494,-0.8301,13.1083,aminoguanidine,GSE70138,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCLB003_PC3_24H_X3_B13:P20,0.3662,-2.1044,0.2024,0.8759,-0.9267,-0.0539,-0.1612,2.8925,0.372,-0.0475,...,0.0881,-2.5475,0.4033,1.1583,0.6428,-1.2056,-0.2261,wortmannin,GSE92742,2166
PCLB003_PC3_24H_X3_B13:P21,0.3775,-0.4531,-0.0109,-0.266,-0.1719,0.0985,-0.1459,-2.066,-0.375,-1.2346,...,-2.0991,-1.4825,2.1151,0.0,0.5136,1.1579,-0.087,wortmannin,GSE92742,2166
PCLB003_PC3_24H_X3_B13:P22,0.5546,0.0,-0.3511,1.6628,-0.8406,-0.6109,-0.4939,-1.4338,0.0589,0.0729,...,-0.2673,-1.2113,-1.0933,1.1583,-0.3659,-0.1505,0.4603,wortmannin,GSE92742,2166
PCLB003_PC3_24H_X3_B13:P23,0.3037,-0.666,1.176,0.5265,-1.0765,0.36,0.4682,7.2672,-0.4834,-0.5445,...,-2.7238,-0.7698,0.3769,0.2699,0.2177,-1.0949,0.3538,wortmannin,GSE92742,2166


# Make train data

In [88]:
before_standardization_GSE70138=expression_freq_pert_df[expression_freq_pert_df['LINCS']=='GSE70138']
before_standardization_GSE92742=expression_freq_pert_df[expression_freq_pert_df['LINCS']=='GSE92742']

In [89]:
X_data_GSE70138=np.array(before_standardization_GSE70138.iloc[:,:-3], dtype=np.float)
y_data_GSE70138=np.array(before_standardization_GSE70138.iloc[:,-1], dtype=np.int)
X_data_GSE92742=np.array(before_standardization_GSE92742.iloc[:,:-3], dtype=np.float)
y_data_GSE92742=np.array(before_standardization_GSE92742.iloc[:,-1], dtype=np.int)

# Perform standardization

In [90]:
# only standardization separately
standard_scaler_GSE70138=StandardScaler()
X_data_standardized_GSE70138=standard_scaler_GSE70138.fit_transform(X_data_GSE70138)
standard_scaler_GSE92742=StandardScaler()
X_data_standardized_GSE92742=standard_scaler_GSE92742.fit_transform(X_data_GSE92742)

# Concat X and y again

In [91]:
expression_pert_np_copied_GSE70138=np.concatenate((X_data_standardized_GSE70138, y_data_GSE70138[:,np.newaxis]), axis=1)
expression_pert_np_copied_GSE92742=np.concatenate((X_data_standardized_GSE92742, y_data_GSE92742[:,np.newaxis]), axis=1)

In [92]:
display(expression_pert_np_copied_GSE70138)
display(expression_pert_np_copied_GSE92742)

array([[ 1.37150971e-01, -2.07043086e-01,  5.39100324e-01, ...,
        -8.58480970e-01, -1.65062948e-01,  0.00000000e+00],
       [-2.21580880e-01, -1.59284215e-01, -1.93752641e-01, ...,
        -4.48627846e-01,  7.55438498e+00,  0.00000000e+00],
       [-3.49563866e-01, -2.65645298e-01,  1.37300374e-01, ...,
         1.24254831e+00,  6.81381403e+00,  0.00000000e+00],
       ...,
       [ 2.59741139e+00,  6.94496038e-01,  2.00999682e+00, ...,
        -1.57371613e-01,  2.80307230e+00,  1.56900000e+03],
       [ 4.40062401e-01,  7.09387554e-01, -1.06916602e+00, ...,
        -9.49833758e-01,  6.37619607e-01,  1.34400000e+03],
       [-9.26925644e-01, -1.26320470e-01,  1.63948191e+00, ...,
         5.58807127e-01,  7.27008605e-01,  3.11000000e+02]])

array([[-6.91001692e-01, -4.59485637e-01, -5.30208931e-01, ...,
        -3.34990409e-01,  8.81371731e-01,  1.74600000e+03],
       [-9.76596940e-01, -1.33826896e-01, -3.22231416e+00, ...,
        -2.17024045e-01,  4.74509878e-02,  1.74600000e+03],
       [-1.40791325e-01, -4.89044246e-01, -2.16662512e-01, ...,
        -9.50907733e-01,  3.05056769e-01,  1.74600000e+03],
       ...,
       [ 2.48438724e-01, -9.51179728e-02, -2.39079956e-01, ...,
        -2.37958915e-01,  3.15439111e-01,  2.16600000e+03],
       [ 8.13308879e-02, -4.37483748e-01,  8.98252030e-01, ...,
        -1.02251830e+00,  2.64718060e-01,  2.16600000e+03],
       [ 5.38629572e-01, -6.58993362e-01,  1.42165697e-01, ...,
        -1.42755079e-01,  1.31585825e+00,  2.16600000e+03]])

# Data concatenation

In [93]:
expression_pert_np_copied=np.concatenate((expression_pert_np_copied_GSE70138,expression_pert_np_copied_GSE92742),axis=0)

# Data shape check

In [94]:
expression_pert_np_copied.shape

(680909, 978)

# For fast test

In [95]:
fast_test_number=10000

In [96]:
expression_pert_np_copied_small=expression_pert_np_copied[:fast_test_number,:]

In [97]:
expression_pert_np_copied_small.shape

(10000, 978)

In [105]:
configurable_parameters_dict['number_of_perturbagens']=len(np.unique(expression_pert_np_copied_small[:fast_test_number,-1]))
# configurable_parameters_dict

In [106]:
configurable_parameters_dict['number_of_perturbagens']=len(np.unique(expression_pert_np_copied[:,-1]))
# configurable_parameters_dict

In [107]:
# y_train_test=expression_pert_np_copied_small[:fast_test_number,-1].astype(int)
y_train_test=expression_pert_np_copied[:,-1].astype(int)

class_counts = np.bincount(y_train_test)
n_samples = len(y_train_test)
total_samples = np.sum(class_counts)
class_weights = (1 / n_samples) * (total_samples / class_counts)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
configurable_parameters_dict['class_weight_dict']=class_weight_dict

In [108]:
print(list(class_weight_dict.items())[:15])

[(0, 0.008064516129032258), (1, 0.007936507936507938), (2, 0.0034482758620689655), (3, 0.007936507936507938), (4, 0.0037735849056603774), (5, 0.003424657534246575), (6, 0.00411522633744856), (7, 0.007936507936507938), (8, 0.0053475935828877), (9, 0.00015898251192368842), (10, 0.007462686567164179), (11, 0.007142857142857143), (12, 0.0029761904761904765), (13, 0.0034722222222222225), (14, 0.008)]


# Main script

In [109]:
torch.cuda.empty_cache()

In [110]:
GET_LIST_OF_CORRECT_AND_INCORRECT_CASES_FOR_CAM=False
ROC_WITH_CI=False
MULTIPLE_MODEL_ROC_IN_ONE_PLOT=False
SUBGROUP_ROC_ANALYSIS=False
CI_IN_PERFORMANCES_IN_SUBGROUPS=True

MODE="inference" # train, inference

DATA_SIZE="full" # full, partial

In [111]:
if __name__=='__main__':
  # ================================================================================
  # Create objects
  
  param_super_class_obj=param_class_module.Param_Super_Class(configurable_parameters_dict)
  utils_data_processing_obj=data_processing_module.Utils_Data_Processing(configurable_parameters_dict)
  train_class_obj=train_module.Train_Class(configurable_parameters_dict)
  test_class_obj=test_model_module.Test_Class(configurable_parameters_dict)
  
  # ================================================================================
  if MODE=='train':
    if DATA_SIZE=="partial":
      print(train_class_obj.train_start(expression_pert_np_copied_small))
    elif DATA_SIZE=="full":
      print(train_class_obj.train_start(expression_pert_np_copied))
  elif MODE=='inference':
    if DATA_SIZE=="partial":
      predictions_df=test_class_obj.test_start(expression_pert_np_copied_small)
    elif DATA_SIZE=="full":
      predictions_df=test_class_obj.test_start(expression_pert_np_copied)

    def l2_normalize(row):
        norm=np.linalg.norm(row)
        return row/norm

    predictions_df = predictions_df.apply(l2_normalize, axis=1)

    # ================================================================================
    # Add drug label
    
    if DATA_SIZE=="partial":
      predictions_df['drug']=expression_pert_np_copied_small[:,-1]
    elif DATA_SIZE=="full":
      predictions_df['drug']=expression_pert_np_copied[:,-1]

    # ================================================================================
    d={'drug_num':list(dictionary.values()),
       'drug_string':list(dictionary.keys())}
    df=pd.DataFrame(d)

    finale_pred=predictions_df.merge(df,left_on='drug',right_on='drug_num',how='inner')
    print('finale_pred',finale_pred)

    with open('./result_files/embedding_vectors.pkl','wb') as f:
      pickle.dump(finale_pred,f)

finale_pred                0         1         2         3         4         5         6  \
0      -0.195774  0.208246 -0.200245 -0.187345  0.298707  0.040652  0.061265   
1      -0.206023  0.269305 -0.164176 -0.130092  0.278392  0.169211  0.038797   
2      -0.229680  0.280046 -0.207327 -0.124709  0.297977  0.150674  0.027637   
3      -0.180996  0.208216 -0.170820 -0.186510  0.247666  0.029806  0.105625   
4      -0.292073  0.245195 -0.254652 -0.015646  0.319038  0.075625  0.082516   
...          ...       ...       ...       ...       ...       ...       ...   
680904 -0.078206 -0.067493 -0.127657  0.138919  0.194810 -0.251248  0.218485   
680905 -0.116378  0.009478 -0.121621  0.184124  0.091733 -0.154005  0.251708   
680906 -0.182747  0.154614 -0.137723  0.249791  0.183348 -0.165592  0.280881   
680907 -0.214649  0.059514 -0.125397  0.211409  0.172681 -0.116643  0.209058   
680908 -0.138644  0.064946 -0.115373  0.216691  0.111257 -0.080148  0.197466   

               7         8 